In [1]:
!pip install findspark
import findspark
findspark.init()

In [2]:
from datetime import datetime
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, DateType

In [3]:
from pyspark.sql import SparkSession
import os

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.13.0 pyspark-shell'

spark_session = SparkSession\
    .builder\
    .getOrCreate()

In [4]:
spark_session

In [5]:
 sc = spark_session._sc

In [6]:
!head /mnt/data/programming-languages.csv

## Read csv file

In [7]:
prog_lang_schema = StructType([
    StructField("name", StringType(), False),
    StructField("wikipedia_url", StringType(), False),
])

In [18]:
prog_langs_df = spark_session.read.csv("file:///mnt/data/programming-languages.csv", schema=prog_lang_schema)

In [19]:
prog_langs_df

DataFrame[name: string, wikipedia_url: string]

In [23]:
prog_langs_df.head()

Row(name='name', wikipedia_url='wikipedia_url')

In [24]:
prog_langs_list = [str(row[0]) for row in prog_langs_df.select('name').collect()]

In [27]:
prog_langs_list[:4]

['name', 'A# .NET', 'A# (Axiom)', 'A-0 System']

## Read xml file

In [36]:
!head -n 4 /mnt/data/posts_sample.xml

In [31]:
posts_sample_df = spark_session.read.format("xml").options(rowTag="row").load('file:///mnt/data/posts_sample.xml')

In [32]:
posts_sample_df.head()

Row(_AcceptedAnswerId=7, _AnswerCount=13, _Body="<p>I want to use a track-bar to change a form's opacity.</p>\n\n<p>This is my code:</p>\n\n<pre><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n\n<p>When I build the application, it gives the following error:</p>\n\n<blockquote>\n  <p>Cannot implicitly convert type <code>'decimal'</code> to <code>'double'</code></p>\n</blockquote>\n\n<p>I tried using <code>trans</code> and <code>double</code> but then the control doesn't work. This code worked fine in a past VB.NET project.</p>\n", _ClosedDate=None, _CommentCount=2, _CommunityOwnedDate=datetime.datetime(2012, 10, 31, 16, 42, 47, 213000), _CreationDate=datetime.datetime(2008, 7, 31, 21, 42, 52, 667000), _FavoriteCount=48, _Id=4, _LastActivityDate=datetime.datetime(2019, 7, 19, 1, 39, 54, 173000), _LastEditDate=datetime.datetime(2019, 7, 19, 1, 39, 54, 173000), _LastEditorDisplayName='Rich B', _LastEditorUserId=3641067, _OwnerDisplayName=None, _OwnerUse

In [37]:
def define_language(row):
    language_tag = None
    for lang in prog_langs_list:
        if '<' + lang.lower() + '>' in row._Tags.lower():
            language_tag = lang
            break
    if language_tag is None:
        return None
    return (row._Id, language_tag)

In [38]:
def check_date_range(row):
    left_border = datetime(year=2010, month=1, day=1)
    right_border = datetime(year=2020, month=1, day=1)
    created_date = row._CreationDate
    return created_date > left_border and created_date < right_border

In [69]:
# 1. Select rows with not null tag and an appropriate date
# 2. (xml ...) -> (id, tag)
# 3. Remove rows with tags without language
# 4. [(id, language)] -> {language: (id, language)}
# 5. {language: n_samples}
top_languages = posts_sample_df.rdd\
    .filter(lambda row: row._Tags is not None and check_date_range(row))\
    .map(define_language)\
    .filter(lambda row: row is not None)\
    .keyBy(lambda row: row[1])\
    .aggregateByKey(
        0,
        lambda acc, value: acc + 1,
        lambda acc1, acc2: acc1 + acc2,
    )\
    .sortBy(lambda row: row[1], ascending=False)\
    .toDF()

In [57]:
top_languages.take(10)

[Row(_1='JavaScript', _2=1799),
 Row(_1='Java', _2=1518),
 Row(_1='PHP', _2=1155),
 Row(_1='Python', _2=1136),
 Row(_1='R', _2=313),
 Row(_1='C', _2=291),
 Row(_1='Objective-C', _2=262),
 Row(_1='Ruby', _2=178),
 Row(_1='Bash', _2=103),
 Row(_1='Scala', _2=92)]

In [70]:
top_languages = top_languages.select(col('_1').alias('ProgrammingLanguage'), col('_2').alias('NumberOfPosts'))

In [79]:
top_languages = sc.parallelize(top_languages.take(10)).toDF()

In [81]:
top_languages.show()

+-------------------+-------------+
|ProgrammingLanguage|NumberOfPosts|
+-------------------+-------------+
|         JavaScript|         1799|
|               Java|         1518|
|                PHP|         1155|
|             Python|         1136|
|                  R|          313|
|                  C|          291|
|        Objective-C|          262|
|               Ruby|          178|
|               Bash|          103|
|              Scala|           92|
+-------------------+-------------+



In [86]:
top_languages.write.parquet("top_languages.parquet")